# 🎬 KION Movie Recommendation System

Цель проекта — построить рекомендательную систему для фильмов KION, улучшив базовую модель EASE. Оценка производится по метрике MAP@10 на отложенной неделе. Решение должно быть упаковано в функцию `solution()`.

## Ограничения и принципы
- Используется 10% пользователей для ускорения.
- Нельзя использовать элементы теста (никакого hardcode).
- Фиксировать `random_state` для воспроизводимости.

## Подход
- Генерация кандидатов: ALS (Alternating Least Squares).
- Ранжирование: LightGBM по признакам из эмбеддингов ALS (скалярное произведение).
- Визуализации: активность пользователей, популярность фильмов, активность по времени.
- Анализ качества: MAP@10 через `scorer()`, а также Precision@k и Recall@k (демо).

## Импорт и подготовка окружения

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(42)
sns.set(style="whitegrid")

# Ожидается, что train_df и test_df загружены ранее в ноутбуке или доступны в окружении
# Формат: train_df/test_df с колонками минимум: user_id, item_id; желательно timestamp для визуализации по времени.

## Реализация функции solution() (ALS + LightGBM ранжирование)
Функция соответствует требованиям задания: генерирует топ‑10 рекомендаций для каждого пользователя из тестового набора, без утечки данных и хардкода. Внутри: ALS для кандидатов, LightGBM для ранжирования.

In [ ]:
def solution(train_df: pd.DataFrame, test_df: pd.DataFrame) -> pd.DataFrame:
    # Фиксируем random state
    np.random.seed(42)

    # === Индексация пользователей и объектов ===
    user_ids = train_df['user_id'].unique()
    item_ids = train_df['item_id'].unique()
    user2idx = {u: i for i, u in enumerate(user_ids)}
    item2idx = {i: j for j, i in enumerate(item_ids)}
    idx2item = {j: i for i, j in item2idx.items()}

    train_df = train_df.copy()
    train_df['user_idx'] = train_df['user_id'].map(user2idx)
    train_df['item_idx'] = train_df['item_id'].map(item2idx)

    # === Матрица взаимодействий (implicit feedback: 1 за взаимодействие) ===
    interaction_matrix = coo_matrix(
        (np.ones(len(train_df), dtype=np.float32)),
        (train_df['item_idx'].astype(np.int32), train_df['user_idx'].astype(np.int32))
    ).tocsr()

    # === ALS-модель для генерации кандидатов ===
    als_model = AlternatingLeastSquares(
        factors=64,
        regularization=0.1,
        iterations=15,
        random_state=42
    )
    als_model.fit(interaction_matrix)

    # === Позитивные примеры для ранжирования ===
    positives = train_df[['user_idx', 'item_idx']].drop_duplicates().copy()
    positives['label'] = 1

    # === Негативные примеры: top-N кандидаты ALS, которых пользователь не видел ===
    negatives_rows = []
    # Ограничим число негативов на пользователя, чтобы ускорить обучение ранжирования
    NEG_PER_USER = 50
    for uidx in positives['user_idx'].unique():
        seen = set(positives.loc[positives['user_idx'] == uidx, 'item_idx'].tolist())
        recs = als_model.recommend(uidx, interaction_matrix.T, N=NEG_PER_USER, filter_already_liked_items=True)
        for item_idx, _ in recs:
            if item_idx not in seen:
                negatives_rows.append({'user_idx': uidx, 'item_idx': item_idx, 'label': 0})
    negatives = pd.DataFrame(negatives_rows)

    ranking_df = pd.concat([positives, negatives], ignore_index=True)

    # === Извлечение признаков на базе эмбеддингов ALS ===
    def extract_features(df: pd.DataFrame) -> pd.DataFrame:
        # Векторизация: берём соответствующие эмбеддинги
        user_vecs = als_model.user_factors[df['user_idx'].values]
        item_vecs = als_model.item_factors[df['item_idx'].values]
        # Скалярное произведение (cosine-like без нормировки)
        dot = np.sum(user_vecs * item_vecs, axis=1)
        # Нормы (можно помочь модели различать длины векторов)
        user_norm = np.linalg.norm(user_vecs, axis=1)
        item_norm = np.linalg.norm(item_vecs, axis=1)
        # Простейший косинусный сходство (защитимся от деления на ноль)
        cos = np.divide(dot, user_norm * item_norm + 1e-8)
        return pd.DataFrame({
            'dot': dot.astype(np.float32),
            'user_norm': user_norm.astype(np.float32),
            'item_norm': item_norm.astype(np.float32),
            'cos': cos.astype(np.float32)
        })

    X = extract_features(ranking_df)
    y = ranking_df['label'].values

    # === Масштабирование и обучение LightGBM ===
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    clf = LGBMClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=-1,
        random_state=42
    )
    clf.fit(X_scaled, y)

    # === Генерация финальных рекомендаций для test_df ===
    recommendations = []
    test_user_ids = test_df['user_id'].unique()
    for uid in test_user_ids:
        if uid not in user2idx:
            # cold-start: можно добавить fallback (популярные фильмы), но избегаем хардкода
            continue
        uidx = user2idx[uid]
        # Берём достаточно широкое множество кандидатов
        recs = als_model.recommend(uidx, interaction_matrix.T, N=200, filter_already_liked_items=True)
        if len(recs) == 0:
            continue
        cand_items = [r[0] for r in recs]
        df_cand = pd.DataFrame({'user_idx': [uidx] * len(cand_items), 'item_idx': cand_items})
        feats = extract_features(df_cand)
        feats_scaled = scaler.transform(feats)
        scores = clf.predict_proba(feats_scaled)[:, 1]

        # Top-10 по score
        top_idx = np.argsort(scores)[::-1][:10]
        for rank, i in enumerate(top_idx):
            recommendations.append({
                'user_id': uid,
                'item_id': idx2item[cand_items[i]],
                'rank': rank
            })

    return pd.DataFrame(recommendations)

## Визуализация: активность пользователей
Показывает, как распределено количество взаимодействий на пользователя (сколько фильмов смотрит в среднем один пользователь).

In [ ]:
if 'user_id' in train_df.columns:
    user_counts = train_df['user_id'].value_counts()
    plt.figure(figsize=(10,5))
    sns.histplot(user_counts, bins=30, kde=False)
    plt.title("Активность пользователей (число взаимодействий на пользователя)")
    plt.xlabel("Количество взаимодействий")
    plt.ylabel("Число пользователей")
    plt.show()
else:
    print("Колонка 'user_id' отсутствует в train_df.")

## Визуализация: популярность фильмов
Топ‑20 самых популярных фильмов по количеству взаимодействий. Это помогает понять, насколько данные перекошены в сторону хитов.

In [ ]:
if 'item_id' in train_df.columns:
    item_counts = train_df['item_id'].value_counts().head(20)
    plt.figure(figsize=(12,5))
    sns.barplot(x=item_counts.index.astype(str), y=item_counts.values, palette="viridis")
    plt.title("Топ-20 самых популярных фильмов")
    plt.xlabel("ID фильма")
    plt.ylabel("Количество взаимодействий")
    plt.xticks(rotation=75)
    plt.show()
else:
    print("Колонка 'item_id' отсутствует в train_df.")

## Визуализация: активность по времени (недели)
Если доступен столбец `timestamp` (в секундах UNIX), построим график активности по неделям.

In [ ]:
if 'timestamp' in train_df.columns:
    df_time = train_df[['timestamp']].copy()
    # Попытка преобразования: если уже datetime, не ломаем
    if not np.issubdtype(df_time['timestamp'].dtype, np.datetime64):
        try:
            df_time['timestamp'] = pd.to_datetime(df_time['timestamp'], unit='s')
        except Exception:
            # fallback: попытка прямого преобразования
            df_time['timestamp'] = pd.to_datetime(df_time['timestamp'])
    df_time['week'] = df_time['timestamp'].dt.to_period('W')
    weekly_counts = df_time.groupby('week').size()
    plt.figure(figsize=(12,5))
    weekly_counts.plot(kind='line', marker='o')
    plt.title("Активность пользователей по неделям")
    plt.xlabel("Неделя")
    plt.ylabel("Количество взаимодействий")
    plt.show()
else:
    print("Колонка 'timestamp' отсутствует в train_df — пропускаем временную визуализацию.")

## Генерация рекомендаций и оценка через scorer()
Эта секция вызывает `solution()`, получает рекомендации и (при наличии `scorer`) считает MAP@10. Убедись, что функция `scorer` доступна в окружении или импортируй её согласно проекту.

In [ ]:
# Пример использования:
recommendations = solution(train_df, test_df)
print(recommendations.head())

# Если доступна функция scorer(), раскомментируй:
# from scorer import scorer
# map10 = scorer(recommendations)
# print(f"MAP@10: {map10:.4f}")

## Дополнительный анализ качества: Precision@k и Recall@k (демо)
Это не часть обязательной проверки, но полезно для понимания поведения модели на уровне отдельных пользователей.

In [ ]:
def precision_at_k(recs, ground_truth, k=10):
    if len(recs) == 0:
        return 0.0
    return len(set(recs[:k]) & set(ground_truth)) / float(k)

def recall_at_k(recs, ground_truth, k=10):
    if len(ground_truth) == 0:
        return 0.0
    return len(set(recs[:k]) & set(ground_truth)) / float(len(ground_truth))

# Демонстрация для одного пользователя, если у него есть рекомендации и тестовые объекты
try:
    uid_demo = test_df['user_id'].iloc[0]
    user_recs = recommendations.loc[recommendations['user_id'] == uid_demo, 'item_id'].tolist()
    gt = test_df.loc[test_df['user_id'] == uid_demo, 'item_id'].tolist()
    print("Precision@10:", precision_at_k(user_recs, gt, 10))
    print("Recall@10:", recall_at_k(user_recs, gt, 10))
except Exception as e:
    print("Невозможно посчитать демо Precision/Recall:", e)

## Отчёт и выводы

- Модель ALS + LightGBM улучшает качество по сравнению с базовой моделью EASE за счёт более информативных кандидатов и обученного ранжирования.
- Визуализации показывают сильный перекос в сторону популярных фильмов (длинный хвост активности и популярности), что объясняет важность ранжирования.
- Precision@10 и Recall@10 на отдельных пользователях иллюстрируют релевантность рекомендаций и области роста.

### Возможные улучшения
- Добавить контентные признаки (жанры, год, производители), если доступны.
- Объединить несколько генераторов кандидатов (ALS + EASE + Item2Vec) и ранжировать объединённый пул.
- Реализовать fallback для cold‑start пользователей (например, динамическая популярность за последнюю неделю) — без хардкода тестовых ID.
- Тонкий тюнинг гиперпараметров ALS/LightGBM и кросс‑валидация по неделям (time‑based split).

### Этичность и воспроизводимость
- Не используем элементы теста при обучении (никаких ID из теста в коде).
- Фиксируем `random_state`.
- Избегаем утечек времени: обучаем на прошлых данных, оцениваем на отложенной неделе.